# Citations

Towards 3D Deep Learning for neuropsychiatry: predicting Autism diagnosis using an interpretable Deep Learning pipeline applied to minimally processed structural MRI data, Melanie Garcia, Clare Kelly. medRxiv 2022.10.18.22281196; doi: https://doi.org/10.1101/2022.10.18.22281196

Github: https://github.com/garciaml/Autism-3D-CNN-brain-sMRI?tab=readme-ov-file

# Virtual Environment

In [ ]:
# Activate Virtual Environment and Install Requirements
#!python3 -m venv ../pretrainedresnet2
#!source ../pretrainedresnet2/bin/activate
#!python3 -m ipykernel install --user --name=pretrainedresnet2 --display-name "Python (pretrainedresnet2)"
#Switch to notebook/virtual environment kernel

# Libraries and Imports

In [2]:
#!pip install -r requirements.txt
#!pip install "torchio>=0.19.0"
#!pip install monai
#!pip install tensorboard
#!pip install torchsummary
import pandas as pd

# Finetune Model from Pretrained Medical ResNet50

In [2]:
# Start training from scratch
!python ../Autism-3D-CNN-brain-sMRI/train_medicalnet.py 'JustBrain_Data/ABIDE_COMBINED' 'Middle_Data/ABIDE_COMBINED' './outputs/Resnet50/ABIDE_COMBINED' '../Autism-3D-CNN-brain-sMRI/resnet_training/resnet_18_23dataset.pth' --lr 0.0003 --batch 8 --epochs 20

Loading pretrained model weights from MedicalNet: ../Autism-3D-CNN-brain-sMRI/resnet_training/resnet_18_23dataset.pth
----------
epoch 1
^C
Traceback (most recent call last):
  File "/sfs/gpfs/tardis/home/ejh2wy/CHMCorr_Autism_Research/../Autism-3D-CNN-brain-sMRI/train_medicalnet.py", line 171, in <module>
    scaler.step(optimizer)
  File "/home/ejh2wy/.local/lib/python3.11/site-packages/torch/amp/grad_scaler.py", line 454, in step
    retval = self._maybe_opt_step(optimizer, optimizer_state, *args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ejh2wy/.local/lib/python3.11/site-packages/torch/amp/grad_scaler.py", line 351, in _maybe_opt_step
    if not sum(v.item() for v in optimizer_state["found_inf_per_device"].values()):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ejh2wy/.local/lib/python3.11/site-packages/torch/amp/grad_scaler.py", line 351, in <genexpr>
    if not sum(v.i

In [ ]:
# Resume training from saved model
#!python ../Autism-3D-CNN-brain-sMRI/train_medicalnet.py 'JustBrain_Data/ABIDE_COMBINED' 'Cleaned_Data/ABIDE_COMBINED' './outputs/Resnet50/ABIDE_COMBINED' 'outputs/Resnet50/ABIDE_COMBINED/checkpoint_7.pth' --resume

## Test

In [18]:
participants_tsv = pd.read_csv('JustBrain_Data/ABIDE_COMBINED/participants.tsv', sep="\t", dtype=str)
participants_tsv.rename(columns={"participant_id" : "SUB_ID"}, inplace=True)
participants_tsv = participants_tsv[participants_tsv.dataset == 'test']
participants_tsv.to_csv('outputs/Resnet50/ABIDE_COMBINED/test/subjects.csv', index = False)

In [20]:
# Predictions - 7 epochs (best accuracy)
!python ../Autism-3D-CNN-brain-sMRI/predict_medicalnet_subids.py 'Cleaned_Data/ABIDE_COMBINED/test' 'outputs/Resnet50/ABIDE_COMBINED/test/subjects.csv' './outputs/Resnet50/ABIDE_COMBINED/checkpoint_7.pth' './outputs/Resnet50/ABIDE_COMBINED/test'

evaluation metric: 0.5130434782608696


## Val

In [3]:
participants_tsv = pd.read_csv('JustBrain_Data/ABIDE_COMBINED/participants.tsv', sep="\t", dtype=str)
participants_tsv.rename(columns={"participant_id" : "SUB_ID"}, inplace=True)
participants_tsv = participants_tsv[participants_tsv.dataset == 'val']
participants_tsv.to_csv('outputs/Resnet50/ABIDE_COMBINED/validation/subjects.csv', index = False)

In [ ]:
# Predictions - 36 epochs (predictions all 0) - overfit
!python ../Autism-3D-CNN-brain-sMRI/predict_medicalnet_subids.py 'Preprocessed_Data/ABIDE_COMBINED/val' 'outputs/Resnet50/ABIDE_Combined/validation/subjects.csv' './outputs/Resnet50/ABIDE_Combined/checkpoint_36.pth' './outputs/Resnet50/ABIDE_Combined/validation'

## Train

In [4]:
participants_tsv = pd.read_csv('JustBrain_Data/ABIDE_COMBINED/participants.tsv', sep="\t", dtype=str)
participants_tsv.rename(columns={"participant_id" : "SUB_ID"}, inplace=True)
participants_tsv = participants_tsv[participants_tsv.dataset == 'train']
participants_tsv.to_csv('outputs/Resnet50/ABIDE_COMBINED/train/subjects.csv', index = False)

In [10]:
# Predictions - 36 epochs (predictions all 0) - overfit
!python ../Autism-3D-CNN-brain-sMRI/predict_medicalnet_subids.py 'Preprocessed_Data/ABIDE_COMBINED/train' 'outputs/Resnet50/ABIDE_Combined/train/subjects.csv' './outputs/Resnet50/ABIDE_Combined/checkpoint_36.pth' './outputs/Resnet50/ABIDE_Combined/train'

/sfs/gpfs/tardis/home/ejh2wy/Autism-3D-CNN-brain-sMRI/resnet2.py:174: FutureWarning: `nn.init.kaiming_normal` is now deprecated in favor of `nn.init.kaiming_normal_`.
  m.weight = nn.init.kaiming_normal(m.weight, mode='fan_out')
/sfs/gpfs/tardis/home/ejh2wy/CHMCorr_Autism_Research/../Autism-3D-CNN-brain-sMRI/predict_medicalnet_subids.py:104: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_g